There are three types of closures in Ruby: blocks, Procs, and lambdas.
  
# Blocks

Blocks are the simplest type. Generally, they are used whenever you want to perform a task upon something. A very common example would be when you want to do execute code for each member of an Array.

In [1]:
[1, 2, 3].each { |x| puts "#{x} squared is #{x**2}" }

1 squared is 1
2 squared is 4
3 squared is 9


[1, 2, 3]

In [2]:
[1, 2, 3].each do |x|
  puts "#{x} squared is #{x**2}"
end

1 squared is 1
2 squared is 4
3 squared is 9


[1, 2, 3]

Because blocks are closures, variables defined inside them are not available outside.

In [3]:
[1, 2, 3].each do |x|
  foo = x
end

if defined? foo
  puts "'foo' is defined"
else
  puts "'foo' is not defined"
end

'foo' is not defined


... unless the variable was defined in the outer scope

In [4]:
bar = nil

[1, 2, 3].each do |x|
  bar = x
end

if defined? bar
  puts "'bar' is defined"
else
  puts "'bar' is not defined"
end

'bar' is defined


Only one block may be passed to a method - in fact, blocks are very special in this regard. They don't show up in a method's `*args` splat, and must be the last argument. Finally, they are provided to the function separately from other arguments.

The `yield` keyword is used to execute a block inside a function that accepts one.

In [5]:
def takes_a_block(*args)
  puts "#{args.count} args provided:"
  args.each { |x| puts " - #{x}"}
  yield
end

takes_a_block(0, 1, 2) { puts 'this is a block' }

3 args provided:
 - 0
 - 1
 - 2
this is a block


The `yield` keyword can also be used to pass arguments from the method to the block being passed into it. Note that blocks which recieve more or fewer arguments that are defined ignore the mismatch.

In [6]:
def passes_ten_to_a_block
  yield 10
end

passes_ten_to_a_block { |x|
  puts "`x` is #{x}"
}

passes_ten_to_a_block { |x, y|
  puts "`x` is #{x}... but if `y` isn't called, no exception is raised."
}

passes_ten_to_a_block {
  puts "What's an `x`?"
}

`x` is 10
`x` is 10... but if `y` isn't called, no exception is raised.
What's an `x`?


# Procs

Unlike blocks, procs are objects - instances of the `Proc` class. They are executed by calling the `Proc#call` method.

In [7]:
foo_proc = Proc.new { puts "I'm a Proc instance." }

puts foo_proc.class
puts foo_proc.call

Proc
I'm a Proc instance.



Unlike blocks, it's possible to pass multiple procs as arguments to a method.

foo_proc = Proc.new { puts 'foo' }
bar_proc = Proc.new { puts 'bar' }

def choose(one, two, choice)
  case choice
  when 1
    one.call
  when 2
    two.call
  end
end

choose foo_proc, bar_proc, 1
choose foo_proc, bar_proc, 2

In fact... a block passed into a method can be converted to a proc by using the `&` prefix.

In [8]:
def my_func(&block)
  puts "Calling #{block}, instance of #{block.class}."
  puts block.call
end

my_func do
  puts "This is a block, right? It was when I defined it at least."
end

Calling #<Proc:0x007fd5bd9dc2f0@<main>:5>, instance of Proc.
This is a block, right? It was when I defined it at least.



# Lambdas

First off - lambdas are procs, but with a few different behaviors. Here's the simplest way to define a lambda.

In [9]:
lambda { |x| puts x }

#<Proc:0x007fd5bd0455c0@<main>:0 (lambda)>

Unlike procs, lambdas validate the number of arguments passed to it when called.

In [10]:
lam = lambda { |x| puts x }
lam.call

ArgumentError: wrong number of arguments (given 0, expected 1)

A more subtle but very important difference between lambdas and procs is the way the `return` keyword is treated. In a lambda, `return` ends execution of the lambda itself and continues to the next line after it was called. In a proc, `return` first goes back out to the line where it was called, and then *`returns` from that context*.

In [11]:
def call_a_lambda
  my_lambda = lambda {
    puts 'In the lambda'
    return
  }
  puts 'Before calling the lambda'
  my_lambda.call
  puts 'After calling the lambda'
end

call_a_lambda

Before calling the lambda
In the lambda
After calling the lambda


In [12]:
def call_a_proc
  my_proc = Proc.new {
    puts 'In the proc'
    return
  }
  puts 'Before calling the proc'
  my_proc.call
  puts 'After calling the proc'
end

call_a_proc

Before calling the proc
In the proc


If passed a single Array, a lambda will not expand it to fill its argument list - but a proc will.

In [13]:
Proc.new { |a, b, c|
  puts "a=#{a}, b=#{b}, c=#{c}"
}.call [1, 2, 3]

a=1, b=2, c=3


In [14]:
lambda { |a, b, c|
  puts "a=#{a}, b=#{b}, c=#{c}"
}.call([1, 2, 3])

ArgumentError: wrong number of arguments (given 1, expected 3)